In [116]:
import dotenv
import pandas as pd

from typing import List

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [117]:
dotenv.load_dotenv()

True

In [118]:
pd.set_option("display.max_colwidth", None)  # Display full content in each column
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.expand_frame_repr", False)  # Prevent line wrapping

# Utils

In [119]:
class EvaluationCategory(BaseModel):
    value: bool = Field(...)
    details: List[str] = Field(...)


class CodeEvaluation(BaseModel):
    code_correctness: EvaluationCategory = Field(...)
    edge_case_handling: EvaluationCategory = Field(...)
    readability: EvaluationCategory = Field(...)
    context_correctness: EvaluationCategory = Field(...)


parser = JsonOutputParser(pydantic_object=CodeEvaluation)

In [120]:
def evaluation_results(evaluation: CodeEvaluation):
    data = [[
        evaluation["code_correctness"]["value"],
        "\n".join(evaluation["code_correctness"]["details"]),
        evaluation['edge_case_handling']['value'],
        "\n".join(evaluation["edge_case_handling"]["details"]),
        evaluation['readability']['value'],
        "\n".join(evaluation["readability"]["details"]),
        evaluation['context_correctness']['value'],
        "\n".join(evaluation["context_correctness"]["details"]),
    ]]
    return data

# Rules Evaluation

In [121]:
def evaluate_rule(input: str, answer: str, context_prompt: str, type: str) -> list:
    if type != "rule":
        print(input, answer, context_prompt, type)
        raise ValueError("Type must be 'rule'")

    prompt = """
    You are an AI assistant tasked with evaluating C# code written for the Ferryt Low-Code Platform. The platform provides predefined process fields, screens, methods, and objects. Your job is to analyze the code's quality and provide detailed feedback on the following aspects:

    1. **Correctness** (`code_correctness`): 
        - Does the code work as intended, adhering to the functionality described in the user query?
        - Identify any logical issues, bugs, or incorrect operations in the code.

    2. **Edge Case Handling** (`edge_case_handling`):
        - Analyze whether the code properly handles edge cases, such as:
            - Null fields.
            - Invalid or unexpected user inputs.
            - Unavailable process fields or screens.
        - Suggest improvements for better error handling or robustness.

    3. **Readability** (`readability`):
        - Is the code clear, well-structured, and easy to understand?
        - Provide suggestions for improving readability, such as using more concise constructs, clear naming conventions, or comments where needed.

    4. **Correct Use of Context** (`context_correctness`):
        - Evaluate if the code makes **correct** use of the relevant context provided in the Ferryt Low-Code Platform:
            - Does the code use process fields, screens, or methods appropriately to fulfill the described functionality?
            - Are required safeguards in place (e.g., checking `.HasValue` before accessing `.Value`) to prevent runtime errors?
            - Are operations on fields (e.g., `SetVisible`, `SetEditable`, `SetRequired`) and screens (e.g., `Show`, `Hide`) used correctly and aligned with platform guidelines?
            - Is unnecessary or redundant use of context avoided?

        **Important**: The goal is not to use all available context but to ensure the **relevant context is used correctly**.

    **Return only valid JSON in the exact format below. Do not add explanations.**  
    Format:
    ```json
    {{
        "code_correctness": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "edge_case_handling": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "readability": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "context_correctness": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }}
    }}

    ---

    ### Context:
    {context}

    ---


    ### User Input:
    {user_input}

    ---

    ### Generated Code:
    {generated_code}
    """

    prompt = prompt.replace("{", "{{").replace("}", "}}")
    prompt = prompt.format(generated_code=answer, user_input=input, context=context_prompt)
    llm = ChatOpenAI(temperature=0.0, model="gpt-4")
    llm_chain = PromptTemplate.from_template(prompt) | llm | parser
    response = llm_chain.invoke(
        input={
            "question": "Evaluate the code.",
            "user_input": input,
            "context": context_prompt,
            "generated_code": answer,
        }
    )
    return evaluation_results(response)

In [122]:
test_cases = [
    {
        "input": "Ustaw odrzucenie przez wiek na true, jeżeli jeden członek rodziny jest niepełnoletni.",
        "response": """
if (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasValue && x.Wiek.Value < 18))
{
    PF.UR_OdrzuceniePrzezWiek = true;
}
        """,
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Set the age rejection to true if one family member is a minor.
        """,
        "type": "rule",
    },
    {
        "input": "Sprawdź czy w rodzinie istnieje osoba starsza niż 80 lat, jeśli tak zmień składkę podstawową na 100 * ilość osób w rodzinie.",
        "response": """
if (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasValue && x.Wiek.Value > 80))
{
    PF.UR_Skladka = 100 * PF.UR_DaneRodziny_T.Length;
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_OdrzuceniePrzezWiek: Bool
- PF.UR_Skladka: Decimal
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Check if there is a person older than 80 years in the family; if so, change the basic contribution to 100 times the number of people in the family.
""",
        "type": "rule",
    },
    {
        "input": "Ustaw minimalną długość tablicy z danymi rodziny na 1.",
        "response": """
PF.UR_DaneRodziny_T.MinimumSize = 1;
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 Set the minimum length of the family data array to 1.
""",
        "type": "rule",
    },
    {
        "input": "Jeśli kod zniżkowy ma wartość \"UAM5\" to obniż składkę o 5 procent i wyświetl pole ze składką.",
        "response" : """
if (PF.UR_KodZnizkowy.HasValue && PF.UR_KodZnizkowy.Value == "UAM5")
{
    PF.UR_Skladka = PF.UR_Skladka * 0.95M;
    PF.UR_Skladka.SetVisible(true);
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_KodZnizkowy: String
- PF.UR_Skladka: Decimal



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
 If the discount code has a value of "UAM5", reduce the contribution by 5 percent and display the contribution field.
""",
        "type": "rule"
    },
    {
        "input": "Jeśli członek rodziny jest studentem, obniż składkę ubezpieczenia o 10%.",
        "response" : """
for (int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    if (PF.UR_DaneRodziny_T[i].DaneStudenta.NumerIndeksu.HasValue)
    {
        PF.UR_Skladka = PF.UR_Skladka * 0.9M;
        break;
    }
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_Skladka: Decimal
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String


### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.

## Query:
If a family member is a student, reduce the insurance premium by 10%.
""",
        "type": "rule"
    },
    {
        "input": "Wyświetl ekran z danymi rodziny i ustaw wszystkie dane rodziny na edytowalne.",
        "response": """
G.E020_DaneRodziny.Show();
for (int i = 0; i < PF.UR_DaneRodziny_T.Length; i++)
{
    PF.UR_DaneRodziny_T[i].Imie.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Nazwisko.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Wiek.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Miejscowosc.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.Ulica.SetEditable(true);
    PF.UR_DaneRodziny_T[i].Adres.KodPocztowy.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.NumerIndeksu.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Kierunek.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Nazwa.SetEditable(true);
    PF.UR_DaneRodziny_T[i].DaneStudenta.Uczelnia.Typ.SetEditable(true);
}
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

# Universal Prompt for C# Code Generation for Client Data and Screens Management

When writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:

## 1. Available Process Fields:

- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String
- PF.UR_RodzajUbezpieczenia: String
- PF.UR_PoziomUbezpieczenia: String
- PF.UR_DaneRodziny_T[].Imie: String
- PF.UR_DaneRodziny_T[].Nazwisko: String
- PF.UR_DaneRodziny_T[].Wiek: Int
- PF.UR_DaneRodziny_T[].Adres.Miejscowosc: String
- PF.UR_DaneRodziny_T[].Adres.Ulica: String
- PF.UR_DaneRodziny_T[].Adres.KodPocztowy: String
- PF.UR_DaneRodziny_T[].DaneStudenta.NumerIndeksu: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Kierunek: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Nazwa: String
- PF.UR_DaneRodziny_T[].DaneStudenta.Uczelnia.Typ: String
- PF.UR_DaneKlienta_S.Imie: String
- PF.UR_DaneKlienta_S.Nazwisko: String
- PF.UR_DaneKlienta_S.Email: String
- PF.UR_DaneKlienta_S.PESEL: String



### Properties:
Fields have the following properties:
- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.

### Actions:
You can perform these operations on the fields:
- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.

### Arrays:
- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.
    A specific element in the array can be accessed via `[index]` or `.Items(index)`.
- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)

## 2. Available Screens:
- `E010_Powitanie`
- `E020_DaneRodziny`
- `E030_WyborUbezpieczenia`
- `E040_Zgody`
- `E050_Podsumowanie`
- `E060_Odrzucenie`
- `E_techMessage`
- `Tech_SessionEndScreen`
- `Tech_TopScreen`
- `Tech_BottomScreen`
- `Tech_TabsScreen`


Do not use screen variables unless specifically asked to by user.

## Methods:
Screens can be accessed by `G.`.
Screens can be controlled using the following methods:
- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.

## 3. Accessing User Data:
- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,
    `UserID`, etc.
    
## 4. ENV Object:
- ENV gives access to many environment variables like:
    * ActionIndices[0] - points to the element on which the action was last performed (in arrays)
    * ApplicationGuid
    * UserDevice.IsMobile
    * UserDevice.BrowserInfo
    * UserDevice.OperatingSystem etc.
    
## 5. ACTION object
- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,
    `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the
    flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,
    `VisibilityFlagShowAll()`
   
## 6. Example Functionalities:
- Setting field values based on conditions.
- Controlling field visibility, editability and requirements.
- Assigning user data to client data under specific conditions.
- Displaying or hiding screens.

## 7. Notes:
- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.
- Reference to the `.Value` property, may cause an error if the field is null.
    To avoid this, first check that the value is via `.HasValue`.
- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures
    and clear logic so that the code is accessible even to individuals with limited programming skills.
- Do not use comments in the code.

Always remember: respond with code only, unless instructed otherwise.


## Query:
Display a screen with family data and set all family data to editable.
""",
        "type": "rule",
    }
]

In [123]:
columns = [
    "input",
    "response",
    "context",
    "type",
    "CodeCorrectness",
    "CodeCorrectnessDetails",
    "EdgeCaseHandling",
    "EdgeCaseHandlingDetails",
    "Readability",
    "ReadabilityDetails",
    "ContextCorrectness",
    "ContextCorrectnessDetails"
]
total_score_df = pd.DataFrame(columns=columns)
for tc in test_cases:
    res = evaluate_rule(input=tc["input"], answer=tc["response"], context_prompt=tc["context"], type=tc.get('type'))
    res[0] = [tc["input"], tc["response"], tc["context"], tc["type"]] + res[0]
    result_df = pd.DataFrame(res, columns=columns)
    total_score_df = pd.concat([total_score_df, result_df], ignore_index=True)


In [124]:
total_score_df

,input,response,context,type,CodeCorrectness,CodeCorrectnessDetails,EdgeCaseHandling,EdgeCaseHandlingDetails,Readability,ReadabilityDetails,ContextCorrectness,ContextCorrectnessDetails
0,"Ustaw odrzucenie przez wiek na true, jeżeli jeden członek rodziny jest niepełnoletni.",\nif (PF.UR_DaneRodziny_T.Any(x => x.Wiek.HasValue && x.Wiek.Value < 18))\n{\n PF.UR_OdrzuceniePrzezWiek = true;\n}\n,"\nYou're an AI assistant on the Ferryt Low-Code Platform.\n\n# Universal Prompt for C# Code Generation for Client Data and Screens Management\n\nWhen writing C# code, use the available process fields and screens to manage client data and user interface. Follow these guidelines:\n\n## 1. Available Process Fields:\n\n- PF.UR_OdrzuceniePrzezWiek: Bool\n- PF.UR_DaneKlienta_S.Imie: String\n- PF.UR_DaneKlienta_S.Nazwisko: String\n- PF.UR_DaneKlienta_S.Email: String\n- PF.UR_DaneKlienta_S.PESEL: String\n- PF.UR_DaneRodziny_T[].Imie: String\n- PF.UR_DaneRodziny_T[].Nazwisko: String\n- PF.UR_DaneRodziny_T[].Wiek: Int\n\n\n\n### Properties:\nFields have the following properties:\n- `HasValue`, `IsEditable`, `IsRequired`, `IsVisible`, `Value`.\n\n### Actions:\nYou can perform these operations on the fields:\n- `SetEditable(bool)`, `SetVisible(bool)`, `SetRequired(bool)`, `SetNull()`, `GetValueOrDefault(defaultValue)`.\n\n### Arrays:\n- Arrays are complex datatypes that store fields or structures. Their name should end with `_T`.\n A specific element in the array can be accessed via `[index]` or `.Items(index)`.\n- Operations on arrays: Length, MinimumSize (sets minimum size of an array eg. PF.Array.MinimumSize = 1)\n\n## 2. Available Screens:\n- `E010_Powitanie`\n- `E020_DaneRodziny`\n- `E030_WyborUbezpieczenia`\n- `E040_Zgody`\n- `E050_Podsumowanie`\n- `E060_Odrzucenie`\n- `E_techMessage`\n- `Tech_SessionEndScreen`\n- `Tech_TopScreen`\n- `Tech_BottomScreen`\n- `Tech_TabsScreen`\n\n\nDo not use screen variables unless specifically asked to by user.\n\n## Methods:\nScreens can be accessed by `G.`.\nScreens can be controlled using the following methods:\n- `Hide()`, `HideAll()`, `Show()`, `ShowAll()`.\n\n## 3. Accessing User Data:\n- `USER.Current` provides access to properties like `IsAuthenticated`, `UserLogin`, `UserFullName`, `UserEmail`,\n `UserID`, etc.\n \n## 4. ENV Object:\n- ENV gives access to many environment variables like:\n * ActionIndices[0] - points to the element on which the action was last performed (in arrays)\n * ApplicationGuid\n * UserDevice.IsMobile\n * UserDevice.BrowserInfo\n * UserDevice.OperatingSystem etc.\n \n## 5. ACTION object\n- ACTION object has methods like: `ClearDocumentRequirement()`, `ClearFieldRequirement()`,\n `SetFieldsReadOnly()` - disables editable fields, `VisibilityFlagHide(flagName)` - hide all actions with the\n flag set, `VisibilityFlagHideAll()`, `VisibilityFlagShow(flagName)` - show all actions with the flag set`,\n `VisibilityFlagShowAll()`\n \n## 6. Example Functionalities:\n- Setting field values based on conditions.\n- Controlling field visibility, editability and requirements.\n- Assigning user data to client data under specific conditions.\n- Displaying or hiding screens.\n\n## 7. Notes:\n- To assign a value to a specific field, use the '=' symbol, e.g., PF.FieldInt1 = 10.\n- Reference to the `.Value` property, may cause an error if the field is null.\n To avoid this, first check that the value is via `.HasValue`.\n- Write a simple and easy-to-understand code. Avoid using advanced or overly complex syntax. Stick to basic structures\n and clear logic so that the code is accessible even to individuals with limited programming skills.\n- Do not use comments in the code.\n\nAlways remember: respond with code only, unless instructed otherwise.\n\n## Query:\nSet the age rejection to true if one family member is a minor.\n",rule,True,The code correctly checks if any family member is a minor and sets the age rejection to true if so.,True,The code correctly handles the edge case where the age field might b

# Results for rule generation

In [125]:
category_map = {
    'CodeCorrectness': 'Correctness',
    'EdgeCaseHandling': 'Edge Case Coverage',
    'Readability': 'Code Readability',
    'ContextCorrectness': 'Contextual Accuracy'
}

true_counts = total_score_df[['CodeCorrectness', 'EdgeCaseHandling', 'Readability', 'ContextCorrectness']].sum()
result_df = pd.DataFrame({
    'Percentage': (true_counts / len(total_score_df)) * 100
})
result_df.head()

,Percentage
CodeCorrectness,83.333333
EdgeCaseHandling,33.333333
Readability,100.0
ContextCorrectness,83.333333


# Validators evaluation

In [126]:
def evaluate_validator(input: str, answer: str, context_prompt: str, type: str) -> list:
    if type != "validator":
        print(input, answer, context_prompt, type)
        raise ValueError("Type must be 'validator'")

    prompt = """
## JavaScript Validation Code Review Guidelines

You're an AI assistant tasked with evaluating JavaScript validation code written for the Ferryt Low-Code Platform.

Your goal is to provide feedback on the quality of validation code based on the following aspects:

### 1. **Correctness** (`code_correctness`)  
   - Does the validation logic work as expected?  
   - Are there any logical flaws or bugs in the code that might cause incorrect validation?  

### 2. **Edge Case Handling** (`edge_case_handling`)  
   - Does the validation properly account for potential edge cases, such as:  
     - Null or undefined inputs.  
     - Empty strings or invalid characters.  
   - Suggest improvements to handle edge cases more effectively, if needed.  

### 3. **Readability** (`readability`)  
   - Is the validation code clear, concise, and easy to understand?  
   - Provide suggestions to improve clarity, such as simplifying expressions, improving naming conventions, or adding comments if necessary.  

### 4. **Context Use** (`context_correctness`)  
   - Does the code correctly use the context provided by the Ferryt Low-Code Platform, including `arguments.Value` and `arguments.IsValid`?  
   - Is the code free of unnecessary complexity or redundant logic?  

> **Note**: Focus on these areas and provide specific, actionable feedback.

    **Return only valid JSON in the exact format below. Do not add explanations.**  
    Format:
    ```json
    {{
        "code_correctness": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "edge_case_handling": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "readability": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }},
        "context_correctness": {{
            "value": true,
            "details": ["message goes here", "message goes here"]
        }}
    }}

    ---

    ### Context:
    {context}

    ---


    ### User Input:
    {user_input}

    ---

    ### Generated Code:
    {generated_code}
    """

    prompt = prompt.replace("{", "{{").replace("}", "}}")
    prompt = prompt.format(generated_code=answer, user_input=input, context=context_prompt)
    llm = ChatOpenAI(temperature=0.0, model="gpt-4")
    llm_chain = PromptTemplate.from_template(prompt) | llm | parser
    response = llm_chain.invoke(
        input={
            "question": "Evaluate the code.",
            "user_input": input,
            "context": context_prompt,
            "generated_code": answer,
        }
    )
    return evaluation_results(response)

In [127]:
test_cases = [
    {
        "input": "Czy potrafisz napisać walidator NIP, który sprawdza poprawność numeru NIP, uwzględniając liczbę cyfr i format?",
        "response": """
var nip = arguments.Value;
var regex = /^[0-9]{10}$/;

if (regex.test(nip)) {
    var weights = [6, 5, 7, 2, 3, 4, 5, 6, 7];
    var sum = 0;

    for (var i = 0; i < 9; i++) {
        sum += parseInt(nip.charAt(i)) * weights[i];
    }

    var controlNumber = sum % 11;
    arguments.IsValid = controlNumber === parseInt(nip.charAt(9));
} else {
    arguments.IsValid = false;
}
        """,
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

Your task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:

1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.

2. **Syntax Restrictions**:
- **No ES6+ Syntax**: Use only ES5 syntax.
- **No Functions or Methods**: Write standalone code snippets without functions.

3. **Validation Code Requirements**:
- The input to validate is always a string, accessible through `arguments.Value`.
- The code must assign a boolean value to `arguments.IsValid`:
    - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.
- Do not return any values or produce output; only update `arguments.IsValid`.

4. **Code Style**:
- Keep the code simple and avoid complex or "fancy" syntax.

5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.

6. **Unknown Answers**: If you do not know the answer, respond with "I don't know" instead of guessing or making up information.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Can you write an NIP validator that checks the correctness of the NIP number, considering the number of digits and format?
""",
        "type": "validator",
    },
    {
        "input": "Zaimplementuj walidator kodu pocztowego.",
        "response": """
var postalCode = arguments.Value;
var regex = /^\d{2}-\d{3}$/;
arguments.IsValid = regex.test(postalCode);
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

Your task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:

1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.

2. **Syntax Restrictions**:
- **No ES6+ Syntax**: Use only ES5 syntax.
- **No Functions or Methods**: Write standalone code snippets without functions.

3. **Validation Code Requirements**:
- The input to validate is always a string, accessible through `arguments.Value`.
- The code must assign a boolean value to `arguments.IsValid`:
    - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.
- Do not return any values or produce output; only update `arguments.IsValid`.

4. **Code Style**:
- Keep the code simple and avoid complex or "fancy" syntax.

5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.

6. **Unknown Answers**: If you do not know the answer, respond with "I don't know" instead of guessing or making up information.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Implement a postal code validator.
""",
        "type": "validator",
    },
    {
        "input": "Daj mi walidator wieku, który ocenia, czy użytkownik ma wiek mieszczący się w przedziale 18-60 lat.",
        "response": """
var age = parseInt(arguments.Value, 10);
arguments.IsValid = age >= 18 && age <= 60;
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

Your task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:

1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.

2. **Syntax Restrictions**:
- **No ES6+ Syntax**: Use only ES5 syntax.
- **No Functions or Methods**: Write standalone code snippets without functions.

3. **Validation Code Requirements**:
- The input to validate is always a string, accessible through `arguments.Value`.
- The code must assign a boolean value to `arguments.IsValid`:
    - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.
- Do not return any values or produce output; only update `arguments.IsValid`.

4. **Code Style**:
- Keep the code simple and avoid complex or "fancy" syntax.

5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.

6. **Unknown Answers**: If you do not know the answer, respond with "I don't know" instead of guessing or making up information.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Give me an age validator that assesses whether the user's age falls within the range of 18-60 years.
""",
        "type": "validator",
    },
    {
        "input": "Wygeneruj walidator numeru karty kredytowej.",
        "response": """
arguments.IsValid = /^(?:4[0-9]{12}(?:[0-9]{3})?|5[1-5][0-9]{14}|3[47][0-9]{13}|6(?:011|5[0-9]{2})[0-9]{12})$/.test(arguments.Value);
""",
        "context": """
You're an AI assistant on the Ferryt Low-Code Platform.

Your task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:

1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.

2. **Syntax Restrictions**:
- **No ES6+ Syntax**: Use only ES5 syntax.
- **No Functions or Methods**: Write standalone code snippets without functions.

3. **Validation Code Requirements**:
- The input to validate is always a string, accessible through `arguments.Value`.
- The code must assign a boolean value to `arguments.IsValid`:
    - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.
- Do not return any values or produce output; only update `arguments.IsValid`.

4. **Code Style**:
- Keep the code simple and avoid complex or "fancy" syntax.

5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.

6. **Unknown Answers**: If you do not know the answer, respond with "I don't know" instead of guessing or making up information.

Always remember: respond with code only, unless instructed otherwise.

## Query:
Generate credit card number validator.
""",
        "type": "validator",
    }
]

<>:55: SyntaxWarning: invalid escape sequence '\d'
<>:55: SyntaxWarning: invalid escape sequence '\d'
/var/folders/v6/4gqwmsnd2hsdbg_gjmqqgz7r0000gn/T/ipykernel_8475/4184778150.py:55: SyntaxWarning: invalid escape sequence '\d'
  "response": """


In [128]:
columns = [
    "input",
    "response",
    "context",
    "type",
    "CodeCorrectness",
    "CodeCorrectnessDetails",
    "EdgeCaseHandling",
    "EdgeCaseHandlingDetails",
    "Readability",
    "ReadabilityDetails",
    "ContextCorrectness",
    "ContextCorrectnessDetails"
]
total_score_df = pd.DataFrame(columns=columns)
for tc in test_cases:
    res = evaluate_validator(input=tc["input"], answer=tc["response"], context_prompt=tc["context"], type=tc.get('type'))
    res[0] = [tc["input"], tc["response"], tc["context"], tc["type"]] + res[0]
    result_df = pd.DataFrame(res, columns=columns)
    total_score_df = pd.concat([total_score_df, result_df], ignore_index=True)

In [129]:
total_score_df.head()

,input,response,context,type,CodeCorrectness,CodeCorrectnessDetails,EdgeCaseHandling,EdgeCaseHandlingDetails,Readability,ReadabilityDetails,ContextCorrectness,ContextCorrectnessDetails
0,"Czy potrafisz napisać walidator NIP, który sprawdza poprawność numeru NIP, uwzględniając liczbę cyfr i format?","\nvar nip = arguments.Value;\nvar regex = /^[0-9]{10}$/;\n\nif (regex.test(nip)) {\n var weights = [6, 5, 7, 2, 3, 4, 5, 6, 7];\n var sum = 0;\n\n for (var i = 0; i < 9; i++) {\n sum += parseInt(nip.charAt(i)) * weights[i];\n }\n\n var controlNumber = sum % 11;\n arguments.IsValid = controlNumber === parseInt(nip.charAt(9));\n} else {\n arguments.IsValid = false;\n}\n","\nYou're an AI assistant on the Ferryt Low-Code Platform.\n\nYour task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:\n\n1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.\n\n2. **Syntax Restrictions**:\n- **No ES6+ Syntax**: Use only ES5 syntax.\n- **No Functions or Methods**: Write standalone code snippets without functions.\n\n3. **Validation Code Requirements**:\n- The input to validate is always a string, accessible through `arguments.Value`.\n- The code must assign a boolean value to `arguments.IsValid`:\n - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.\n- Do not return any values or produce output; only update `arguments.IsValid`.\n\n4. **Code Style**:\n- Keep the code simple and avoid complex or ""fancy"" syntax.\n\n5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.\n\n6. **Unknown Answers**: If you do not know the answer, respond with ""I don't know"" instead of guessing or making up information.\n\nAlways remember: respond with code only, unless instructed otherwise.\n\n## Query:\nCan you write an NIP validator that checks the correctness of the NIP number, considering the number of digits and format?\n",validator,True,The code correctly checks the NIP number format and calculates the control number.,False,The code does not handle cases where the input is null or undefined. It would throw an error when trying to apply the regex test or charAt method.,True,The code is well-structured and easy to understand. The use of a regex for format checking and a loop for control number calculation is clear.,True,The code correctly uses the arguments.Value for input and arguments.IsValid for output. It does not introduce unnecessary complexity.
1,Zaimplementuj walidator kodu pocztowego.,\nvar postalCode = arguments.Value;\nvar regex = /^\d{2}-\d{3}$/;\narguments.IsValid = regex.test(postalCode);\n,"\nYou're an AI assistant on the Ferryt Low-Code Platform.\n\nYour task is to help users create simple validation logic in JavaScript. Follow these instructions strictly:\n\n1. **Return Only Code**: By default, respond with **only the JavaScript validation code**. Do not add explanations or comments unless the user explicitly asks for an explanation.\n\n2. **Syntax Restrictions**:\n- **No ES6+ Syntax**: Use only ES5 syntax.\n- **No Functions or Methods**: Write standalone code snippets without functions.\n\n3. **Validation Code Requirements**:\n- The input to validate is always a string, accessible through `arguments.Value`.\n- The code must assign a boolean value to `arguments.IsValid`:\n - Set `arguments.IsValid` to `true` if valid, and `false` otherwise.\n- Do not return any values or produce output; only update `arguments.IsValid`.\n\n4. **Code Style**:\n- Keep the code simple and avoid complex or ""fancy"" syntax.\n\n5. **Explain Only When Asked**: If the user requests an explanation or further details, then and only then, provide additional information about the code.\n\n6. **Unknown Answers**: If you do not know the answer, respond with ""I don't know"" instead of guess

In [130]:
category_map = {
    'CodeCorrectness': 'Correctness',
    'EdgeCaseHandling': 'Edge Case Coverage',
    'Readability': 'Code Readability',
    'ContextCorrectness': 'Contextual Accuracy'
}

true_counts = total_score_df[['CodeCorrectness', 'EdgeCaseHandling', 'Readability', 'ContextCorrectness']].sum()
result_df = pd.DataFrame({
    'Percentage': (true_counts / len(total_score_df)) * 100
})
result_df.head()

,Percentage
CodeCorrectness,100.0
EdgeCaseHandling,25.0
Readability,100.0
ContextCorrectness,100.0
